# Simple RNN

### Библиотеки

In [62]:
%pip install numpy
%pip install matplotlib
%pip install scipy==1.11.4

%pip install scikit-learn
%pip install tensorflow
%pip install tensorflow_datasets

%pip install --upgrade gensim

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB 435.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/11.0 MB 653.6 kB/s eta 0:00:17
    --------------------------------------- 0.2/11.0 MB 1.0 MB/s eta 0:00:11
   - -------------------------------------- 0.3/11.0 MB 1.6 MB/s eta 0:00:07
   - -------------------------------------- 0.5/11.0 MB 2.4 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/11.0 MB 3.3 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/11.0 MB 4.2 MB/s

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import setuptools.dist		#Why tensorflow import is not working without this???
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import sklearn.metrics

import gensim.downloader


## Работа с данными

In [ ]:
dataset_train = tf.data.TextLineDataset(["Datasets/hpmor_ru_no_extra_lines.txt"]).filter(lambda x: tf.not_equal(x, "* * *") and tf.not_equal(x, "… ") and tf.not_equal(tf.strings.substr(x, 0, 1), "[") and tf.not_equal(tf.strings.substr(x, 0, 10), "Глава"))
dataset_author_lines_train = dataset_train.filter(lambda x: tf.not_equal(tf.strings.substr(x, 0, 4), "— "))
dataset_characters_lines_train = dataset_train.filter(lambda x: tf.equal(tf.strings.substr(x, 0, 4), "— "))

print(f"Whole dataset shape: {tf.shape(list(dataset_train.as_numpy_iterator()))}")
print(f"Author lines dataset shape: {tf.shape(list(dataset_author_lines_train.as_numpy_iterator()))}")
print(f"Characters lines dataset shape: {tf.shape(list(dataset_characters_lines_train.as_numpy_iterator()))}")

print(f"{os.linesep}Author lines peek:{os.linesep}")
for line in dataset_author_lines_train.take(20):
	print(line.numpy().decode("utf-8"))
print(f"{os.linesep}Character lines peek:{os.linesep}")
for line in dataset_characters_lines_train.take(20):
	print(line.numpy().decode("utf-8"))

Whole dataset shape: [22393]
Author lines dataset shape: [12858]
Characters lines dataset shape: [9535]

Author lines peek:

В лунном свете блестит полоска серебра…
(падают тёмные одежды)
...кровь льётся литрами, и слышен крик. 
Все стены до последнего дюйма заняты книжными шкафами. B каждом шкафу по шесть полок, которые доходят почти до потолка. Некоторые полки плотно заставлены книгами в твёрдом переплёте: математика, химия, история и так далее. На других полках в два ряда стоит научная фантастика в мягкой обложке. Под второй ряд книг подложены коробки и деревянные бруски так, что он выглядывает из-за первого и можно прочитать названия стоящих в нём книг. Но и это не всё: книги перебираются на столы и диваны, образуют небольшие стопки под окнами.
Так выглядит гостиная дома, в котором живут известный профессор Майкл Веррес-Эванс и его жена, миссис Петуния Эванс-Веррес, а также их приёмный сын, Гарри Джеймс Поттер-Эванс-Веррес.
На столе в гостиной лежит письмо, а рядом с ним — желтоват

## Токенизация по словам

### Основные параметры

In [339]:
batch_size = 32
embedding_size = 16

### Создание модели

In [354]:
lines=[]
for x in dataset_author_lines_train:
	lines.append(str(x.numpy(), 'utf-8'))

tokenizer = keras.preprocessing.text.Tokenizer(lower=True)
tokenizer.fit_on_texts(lines)
token_eos_index=len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = token_eos_index
vocabulary_size = token_eos_index+1
tokenizer_items_reverse={}
i=0
for word in tokenizer.word_index.items():
	tokenizer_items_reverse.update({i:word})
	i+=1
print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(tokenizer.word_index.items())

token_sequences = []
y=[]
token_sequence_length_max=0
for line in lines:
	tokens = tokenizer.texts_to_sequences([line])[0]
	for i in range(0, len(tokens)):
		i_next=i+1
		ngram_sequence=tokens[:i_next]
		try:
			y.append(tokens[i_next])
		except IndexError:
			y.append(tokenizer.word_index['<eos>'])

		token_sequences.append(ngram_sequence)
		ngram_sequence_length=len(ngram_sequence)
		if ngram_sequence_length>token_sequence_length_max:
			token_sequence_length_max=ngram_sequence_length
token_sequences = np.array(keras.preprocessing.sequence.pad_sequences(token_sequences, maxlen=token_sequence_length_max, padding="post"))
x, y = token_sequences[:,:-1], np.array(y)
print(x)


Vocabulary size: 39974
dict_items([('и', 1), ('не', 2), ('в', 3), ('Гарри', 4), ('что', 5), ('на', 6), ('с', 7), ('он', 8), ('—', 9), ('его', 10), ('то', 11), ('как', 12), ('это', 13), ('но', 14), ('было', 15), ('из', 16), ('по', 17), ('к', 18), ('бы', 19), ('за', 20), ('она', 21), ('её', 22), ('а', 23), ('Драко', 24), ('от', 25), ('всё', 26), ('чтобы', 27), ('И', 28), ('если', 29), ('же', 30), ('у', 31), ('так', 32), ('ещё', 33), ('был', 34), ('Он', 35), ('они', 36), ('Но', 37), ('когда', 38), ('В', 39), ('о', 40), ('уже', 41), ('даже', 42), ('Гермиона', 43), ('профессор', 44), ('Квиррелл', 45), ('только', 46), ('него', 47), ('ему', 48), ('Профессор', 49), ('для', 50), ('их', 51), ('просто', 52), ('профессора', 53), ('была', 54), ('лишь', 55), ('Она', 56), ('до', 57), ('который', 58), ('или', 59), ('чем', 60), ('все', 61), ('мог', 62), ('время', 63), ('ни', 64), ('себя', 65), ('того', 66), ('были', 67), ('На', 68), ('может', 69), ('раз', 70), ('МакГонагалл', 71), ('которые', 72), ('сн

In [335]:
model_by_words = keras.models.Sequential([
	keras.layers.Embedding(vocabulary_size, embedding_size),
	keras.layers.SimpleRNN(128, return_sequences=True),
	keras.layers.SimpleRNN(128),
	keras.layers.Dense(128, activation='relu'),
	keras.layers.Dense(vocabulary_size, activation='softmax')
])
model_by_words.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model_by_words.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_43 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_43 (SimpleRNN)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_44 (SimpleRNN)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Обучение модели

In [340]:
model_by_words.fit(x, y, batch_size=batch_size)

8685/8685 ━━━━━━━━━━━━━━━━━━━━ 1077s 124ms/step - loss: 8.2982


### Тестирование модели

In [341]:
def decode(sentence_vectorized, map):
	sentence=[]
	for i in sentence_vectorized:
		sentence.append(map[i])
	return ''.join(sentence)

def generateByWords(model, sentence_initial, sentence_generated_length, sequence_length_max, temperature=1.0):
	for i in range(sentence_generated_length):
		sentence_initial_vectorized = keras.preprocessing.sequence.pad_sequences([tokenizer.texts_to_sequences([sentence_initial])[0]], maxlen=sequence_length_max-1, padding='post')
		probs = tf.exp(tf.math.log(model(sentence_initial_vectorized)[0])/temperature).numpy().astype(np.float64)
		probs /= np.sum(probs)
		word_index = np.argmax(np.random.multinomial(1, probs, 1))
		if word_index==token_eos_index:
			break
		sentence_initial+=" "+tokenizer_items_reverse[word_index][0]
	return sentence_initial

In [342]:
sentence_initial="Медленно"
sentence_generated_length=10

In [343]:
print(generateByWords(model_by_words, sentence_initial, sentence_generated_length, token_sequence_length_max))

Медленно временем устройств следующий любящий неё не


### Оценка качества модели

Выборка

In [344]:
sampling_callback = keras.callbacks.LambdaCallback(on_epoch_end = lambda batch, logs: print(os.linesep+generateByWords(model_by_words, sentence_initial, sentence_generated_length, token_sequence_length_max)))
model_by_words.fit(x, y, batch_size=batch_size, callbacks=[sampling_callback], epochs=2)

Epoch 1/2
8685/8685 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 8.0568
Медленно высоко люди её представил домашнюю поразило в признать они очевидной
8685/8685 ━━━━━━━━━━━━━━━━━━━━ 1097s 126ms/step - loss: 8.0568
Epoch 2/2
8685/8685 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 8.0181
Медленно ему пробежали откуда на Гарри чёрный ну она
8685/8685 ━━━━━━━━━━━━━━━━━━━━ 1095s 126ms/step - loss: 8.0181


Температура

In [349]:
for i in range(3):
	temperature=np.random.uniform(0.1, 2.0)
	print(f"Temperature: {temperature}")
	for sentence_initial in ["Гарри", "Гермиона"]:
		print(generateByWords(model_by_words, sentence_initial, sentence_generated_length, token_sequence_length_max, temperature))

Temperature: 1.658132642242083
Гарри столу усиливая оставив картинка убить обеспокоена тормозила тон закончено моральной
Гермиона толкнул рассчитывать конца до Через голосом ярость взял фиолетовых четырьмя
Temperature: 1.6895995016223246
Гарри паролем Ханна царстве» лучше ближайшие слева Забудь рассеялась тонкий балерина
Гермиона написал сове шёл отмеренную сидел гласными придерживались которым затрясла рваных
Temperature: 1.5314169628363747
Гарри ринулся мимо «Приземление оставит точным прямо на сухой
Гермиона промолчал чем


## Посимвольная токенизация

### Оснонвые параметры

In [292]:
batch_size = 16

### Создание модели

In [293]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=False)
texts=[]

for i in dataset_characters_lines_train:
	texts.append(i.numpy().decode('utf-8'))
tokenizer.fit_on_texts(texts)
token_eos_index=len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = token_eos_index
vocabulary_size = token_eos_index + 1

tokenizer_items_reverse={}
for key, val in tokenizer.word_index.items():
	tokenizer_items_reverse.update({val:key})

In [294]:
model_by_symbols = keras.models.Sequential([
	keras.layers.Masking(input_shape=(None, vocabulary_size)),
	keras.layers.SimpleRNN(128, return_sequences=True),
	keras.layers.Dense(vocabulary_size, activation='softmax')
])
model_by_symbols.summary()
model_by_symbols.compile(loss='categorical_crossentropy')

c:\Users\Root\Documents\GitHub\Labs\ai-lab-m3-generative-networks-deepdarkdiffusion\.venv\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, None, 141)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_28 (SimpleRNN)       │ (None, None, 128)      │        34,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, None, 141)      │        18,189 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,749 (206.05 KB)

 Trainable params: 52,749 (206.05 KB)

 Non-trainable params: 0 (0.00 B)

### Обучение модели

In [295]:
def encode_py(x):
	texts = []
	for text in x:
		texts.append(text.numpy().decode('utf-8'))
	texts_vectorized = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(texts))
	return tf.one_hot(texts_vectorized, vocabulary_size), tf.one_hot(tf.concat([texts_vectorized[:,1:], tf.constant(token_eos_index, shape=(len(texts_vectorized), 1))], axis=1), vocabulary_size)

@tf.function
def encode(x):
	a, b=tf.py_function(encode_py, inp=[x], Tout=(tf.float32, tf.float32))
	a.set_shape([None, None, vocabulary_size])
	b.set_shape([None, None, vocabulary_size])
	return a, b

In [296]:
model_by_symbols.fit(dataset_characters_lines_train.batch(batch_size).map(encode))

596/596 ━━━━━━━━━━━━━━━━━━━━ 62s 101ms/step - loss: 1.3626


C:\Program Files\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


### Тестирование модели

In [304]:
def generateBySymbols(model, sentence_initial, sentence_generated_length, temperature=1.0):
		sentence_initial_vectorized = tokenizer.texts_to_sequences([sentence_initial])[0]
		sentence_generated_vectorized = sentence_initial_vectorized
		
		for i in range(sentence_generated_length):
			probs = tf.exp(tf.math.log(model(tf.expand_dims(tf.one_hot(sentence_initial_vectorized, vocabulary_size),0))[0][-1])/temperature).numpy().astype(np.float64)
			probs /= np.sum(probs)
			char_index = np.argmax(np.random.multinomial(1, probs, 1))
			if char_index==token_eos_index:
				break
			sentence_generated_vectorized.append(char_index)
			sentence_initial_vectorized = sentence_initial_vectorized+[char_index]
			
		return decode(sentence_generated_vectorized, tokenizer_items_reverse)

In [315]:
sentence_initial="— "
sentence_generated_length=100

In [318]:
print(generateBySymbols(model_by_symbols, sentence_initial, sentence_generated_length, 0.5))

— Потверу нам пробедном ствериная с вско даверние семен ертально, чтобы и стев иго стого профенсора, к


### Оценка качества модели

Выборка

In [319]:
sampling_callback = keras.callbacks.LambdaCallback(on_epoch_end = lambda batch, logs: print(os.linesep+generateBySymbols(model_by_symbols, sentence_initial, sentence_generated_length)))
model_by_symbols.fit(dataset_characters_lines_train.batch(batch_size).map(encode), callbacks=[sampling_callback], epochs=3)

Epoch 1/3


    596/Unknown 62s 104ms/step - loss: 0.6678

C:\Program Files\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



— Вом жетрам патершенне помопрастотвстеенамовал. Бназа правихмая, что бы помода, асторько дего васшуй 
596/596 ━━━━━━━━━━━━━━━━━━━━ 74s 125ms/step - loss: 0.6678
Epoch 2/3
596/596 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.6498
— Но зюЕстерда, эромо зеблата на Тае ободнов Холла. — Свершесь мертикьго, радена, это веё мажные это н
596/596 ━━━━━━━━━━━━━━━━━━━━ 71s 119ms/step - loss: 0.6498
Epoch 3/3
596/596 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.6347
— по мегиршеге. И прося слажием, что это блетух правзакррика педеё кокой-то сублупиначиснимсе, к комор
596/596 ━━━━━━━━━━━━━━━━━━━━ 70s 118ms/step - loss: 0.6347


Температура

In [320]:
for i in range(5):
	temperature=np.random.uniform(0.1, 2.0)
	print(f"Temperature: {temperature}")
	for sentence_initial in ["Гарри", "Гермиона"]:
		print(generateBySymbols(model_by_symbols, sentence_initial, sentence_generated_length, temperature))

Temperature: 0.4845592727192378
Гарри, что влего, кободом отваться напроверястеренее, не волишь с болько не дожет вошьшь мего стона потве
Гермиона вого прастерние всёб этом обудать в способнал. Ты потама свають я воршка — на может была ворушь мен
Temperature: 1.1668004862705328
Гарри.
Гермиона, обружа-шли па-шинамвет яюби, на дергаецаюство прав дежетсгде? — со, жчить, нето вышом, осмадаланик
Temperature: 0.7857314418089859
Гарри! — сказала сличено делошко в дура не спой то горознота в чимет в севересь, что потченныса квинерил 
Гермиона стому вом произатерьно не оменся голес резучистониль, и ответить Ссебодвенной если я систельно бело
Temperature: 0.5478143951911407
Гарри, — провычастаетерина, не подума и посторать делеть слочко подимають в обы произнать такой всех серо
Гермиона, — сказала профессора Смерти свозмужно, что вы полазать, что он вожет слите прогда пострака меня бы
Temperature: 0.48051791799333554
Гарри правоственность вадела не всё разнейна.
Гермиона сказала сель в обудкаю,